In [1]:
!pwd
!ls

/content
sample_data


In [2]:
import numpy as np
import scipy
import pandas as pd
import math
import random
import sklearn
from nltk.corpus import stopwords
from scipy import spatial
from scipy.sparse import csr_matrix
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse.linalg import svds
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt

In [3]:
from google.colab import drive
drive.mount('/content/drive', force_remount = True)

Mounted at /content/drive


In [4]:
import subprocess
subprocess.run([ 'cp', '-r', '/content/drive/MyDrive/Dataset', '/content/sample_data' ])

CompletedProcess(args=['cp', '-r', '/content/drive/MyDrive/Dataset', '/content/sample_data'], returncode=0)

In [5]:
cd /content/sample_data/Dataset

/content/sample_data/Dataset


In [6]:
!ls .

shared_articles.csv  users_interactions.csv


In [7]:
articles_df = pd.read_csv('shared_articles.csv')
articles_df.head(10)

,timestamp,eventType,contentId,authorPersonId,authorSessionId,authorUserAgent,authorRegion,authorCountry,contentType,url,title,text,lang
0,1459192779,CONTENT REMOVED,-6451309518266745024,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,http://www.nytimes.com/2016/03/28/business/dea...,"Ethereum, a Virtual Currency, Enables Transact...",All of this work is still very early. The firs...,en
1,1459193988,CONTENT SHARED,-4110354420726924665,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,http://www.nytimes.com/2016/03/28/business/dea...,"Ethereum, a Virtual Currency, Enables Transact...",All of this work is still very early. The firs...,en
2,1459194146,CONTENT SHARED,-7292285110016212249,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,http://cointelegraph.com/news/bitcoin-future-w...,Bitcoin Future: When GBPcoin of Branson Wins O...,The alarm clock wakes me at 8:00 with stream o...,en
3,1459194474,CONTENT SHARED,-6151852268067518688,3891637997717104548,-1457532940883382585,NaN,NaN,NaN,HTML,https://cloudplatform.googleblog.com/2016/03/G...,Google Data Center 360° Tour,We're excited to share the Google Data Center ...,en
4,1459194497,CONTENT SHARED,2448026894306402386,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,https://bitcoinmagazine.com/articles/ibm-wants...,"IBM Wants to ""Evolve the Internet"" With Blockc...",The Aite Group projects the blockchain market ...,en
5,1459194522,CONTENT SHARED,-2826566343807132236,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,http://www.coindesk.com/ieee-blockchain-oxford...,IEEE to Talk Blockchain at Cloud Computing Oxf...,One of the largest and oldest organizations fo...,en
6,1459194557,CONTENT SHARED,-2148899391355011268,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,http://www.newsbtc.com/2016/03/28/banks-need-c...,Banks Need To Collaborate With Bitcoin and Fin...,It will take time until banks come around to t...,en
7,1459194599,CONTENT SHARED,4119190424078847945,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,https://bitcoinmagazine.com/articles/blockchai...,Blockchain Technology Could Put Bank Auditors ...,When most people think about computers and rob...,en
8,1459194751,CONTENT SHARED,-7926018713416777892,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,https://news.bitcoin.com/conglomerates-intervi...,Why Decentralized Conglomerates Will Scale Bet...,"Bitcoin.com spoke with the OpenLedger CEO, Ron...",en
9,1459194842,CONTENT SHARED,3353902017498793780,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,https://www.cryptocoinsnews.com/ethereum-rise-...,The Rise And Growth of Ethereum Gets Mainstrea...,"Ethereum, considered by many to be the most pr...",en


In [8]:
articles_df = articles_df[articles_df['eventType'] == 'CONTENT SHARED']

In [9]:
interactions_df = pd.read_csv('users_interactions.csv')
interactions_train = interactions_df[interactions_df['timestamp'] < 1483232400]
interactions_test = interactions_df[interactions_df['timestamp'] >= 1483232400]
print(interactions_train.size)
print(interactions_test.size)

532064
46432


In [10]:
event_type_strength = {
   'VIEW': 1.0,
   'LIKE': 2.0,
   'COMMENT CREATED': 3.0,
   'BOOKMARK': 3.0,
   'FOLLOW': 4.0,
}

In [11]:
def filterData(data):
  data['eventStrength'] = data['eventType'].apply(lambda x : event_type_strength[x])
  interactions_count_df = data.groupby(['personId', 'contentId']).size().groupby('personId').size()
  valid_df = interactions_count_df[interactions_count_df >= 5].reset_index()[['personId']]
  interactions_from_selected_users_df = data.merge(right=valid_df, how='inner', on='personId')
  final_df = interactions_from_selected_users_df.groupby(['personId', 'contentId'])['eventStrength'].sum().apply(lambda x : math.log(1+x, 2)).reset_index()
  return final_df

In [12]:
interactions_df['eventStrength'] = interactions_df['eventType'].apply(lambda x : event_type_strength[x])
interactions_df.sort_values(by='timestamp')

,timestamp,eventType,contentId,personId,sessionId,userAgent,userRegion,userCountry,eventStrength
25389,1457963676,VIEW,8078873160882064481,3891637997717104548,-8752981416571543411,Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/53...,SP,BR,1.0
25390,1457964607,VIEW,8078873160882064481,692689608292948411,-2436141008606551134,Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKi...,SP,BR,1.0
8301,1457966363,VIEW,8078873160882064481,-6411371239765052331,7102678958954943769,Mozilla/5.0 (Windows NT 6.3; Win64; x64) Apple...,TX,US,1.0
4560,1458147079,VIEW,1179326165172129711,-5094446514734487376,-7144255767019066074,Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/53...,SP,BR,1.0
11881,1458148861,VIEW,1179326165172129711,-5299969637554720500,7727337495921885979,Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKi...,SP,BR,1.0
...,...,...,...,...,...,...,...,...,...
69511,1488300723,VIEW,6607431762270322325,-1393866732742189886,2367029511384577082,Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/53...,MG,BR,1.0
71148,1488307874,VIEW,4109618890343020064,3891637997717104548,-7416795577834806518,Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/53...,SP,BR,1.0
71150,1488308005,COMMENT CREATED,4109618890343020064,3891637997717104548,-7416795577834806518,Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/53...,SP,BR,3.0
71149,1488308005,FOLLOW,4109618890343020064,3891637997717104548,-7416795577834806518,Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/53...,SP,BR,4.0


In [13]:
interactions_df[interactions_df['contentId'] < 0].size

324981

In [14]:
person = interactions_df[interactions_df['personId'] == -9223121837663643404]
#person = person[person['contentId'] == -7423191370472335463]
#articles_df.query('contentId == -7423191370472335463')['text']
person['contentId'] == -7423191370472335463

866      False
1475     False
2916     False
5309     False
9221     False
         ...  
68860    False
68888    False
68896    False
69856    False
70760    False
Name: contentId, Length: 63, dtype: bool

In [15]:
users_interactions_count_df = interactions_df.groupby(['personId', 'contentId']).size().groupby('personId').size()
#print(users_interactions_count_df)
users_with_enough_interactions_df = users_interactions_count_df[users_interactions_count_df >= 5].reset_index()[['personId']]
#print(interactions_df.groupby(['personId', 'contentId']).size().groupby('personId').size())
#print(interactions_df.groupby('personId').size())
#users_interactions_count_df[users_interactions_count_df >= 5].reset_index()
#users_with_enough_interactions_df.columns = ['personId', 'interactionsCount']
#users_with_enough_interactions_df

In [16]:
print(len(users_with_enough_interactions_df)) #1.1k
print(len(interactions_df)) #72k
interactions_from_selected_users_df = interactions_df.merge(right=users_with_enough_interactions_df, how='inner', on='personId')
print(len(interactions_from_selected_users_df)) #69k
print(interactions_from_selected_users_df.groupby(['personId', 'contentId'])['eventStrength'].sum().max())
interactions_full_df = interactions_from_selected_users_df.groupby(['personId', 'contentId'])['eventStrength'].sum().apply(lambda x : math.log(1+x, 2)).reset_index()
interactions_full_df
#interactions_full_df

1140
72312
69868
268.0


,personId,contentId,eventStrength
0,-9223121837663643404,-8949113594875411859,1.000000
1,-9223121837663643404,-8377626164558006982,1.000000
2,-9223121837663643404,-8208801367848627943,1.000000
3,-9223121837663643404,-8187220755213888616,1.000000
4,-9223121837663643404,-7423191370472335463,3.169925
...,...,...,...
39101,9210530975708218054,8477804012624580461,3.459432
39102,9210530975708218054,8526042588044002101,1.000000
39103,9210530975708218054,8856169137131817223,1.000000
39104,9210530975708218054,8869347744613364434,1.000000


select interactions_df.*, users_with_enough_interactions_df.personId
from interactions_df
inner join users_with_enough_interactions_df on interactions_df.personId = users_with_enough_interactions_df.personId

In [17]:
interactions_train_df, interactions_test_df = train_test_split(interactions_full_df,
                                   stratify=interactions_full_df['personId'],
                                   test_size=0.20,
                                   random_state=42)

In [18]:
interactions_train = filterData(interactions_train)
interactions_test = filterData(interactions_test)

print(len(interactions_train))
print(len(interactions_test))

35530
2423


<ipython-input-11-b1a7942c8217>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['eventStrength'] = data['eventType'].apply(lambda x : event_type_strength[x])
<ipython-input-11-b1a7942c8217>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['eventStrength'] = data['eventType'].apply(lambda x : event_type_strength[x])


In [19]:
interactions_full_indexed_df = interactions_full_df.set_index('personId')
#print(interactions_full_indexed_df.head(50))
interactions_train_indexed_df = interactions_train_df.set_index('personId')
interactions_test_indexed_df = interactions_test_df.set_index('personId')

In [20]:
def get_items_interacted(person_id, interactions_df):
    # Get the user's data and merge in the movie information.
    interacted_items = interactions_df.loc[person_id]['contentId']
    return set(interacted_items if type(interacted_items) == pd.Series else [interacted_items])

In [92]:
#Top-N accuracy metrics consts
EVAL_RANDOM_SAMPLE_NON_INTERACTED_ITEMS = 100

class ModelEvaluator:

    def __init__(self):
        pass

    #returns a random sample of size sample_size with negative samples
    def get_not_interacted_items_sample(self, person_id, sample_size, seed=42):
        interacted_items = get_items_interacted(person_id, interactions_full_indexed_df)
        all_items = set(articles_df['contentId'])
        non_interacted_items = all_items - interacted_items

        random.seed(seed)
        non_interacted_items_sample = random.sample(non_interacted_items, sample_size)
        return set(non_interacted_items_sample)

    #returns hit (0 if item_id is not found in recommended_items and 1 otherwise), index (index of first occurence of item_id in recommended_items)
    def _verify_hit_top_n(self, item_id, recommended_items, topn):
            try:
                index = next(i for i, c in enumerate(recommended_items) if c == item_id)
            except:
                index = -1
            hit = int(index in range(0, topn))
            return hit, index

    #for a given model and a person_id, this method should return a dictionary with respective metrics for this particular person_id (hits_at_5, hits_at_10, recall_at_5, recall_at_10)
    #they are based on the items, predicted by the model, and the interacted items
    def evaluate_model_for_user(self, model, person_id):
        #Getting the items in test set for this particular person_id
        interacted_values_testset = interactions_test_indexed_df.loc[person_id] # -> dataframe
        if type(interacted_values_testset['contentId']) == pd.Series:
            person_interacted_items_testset = set(interacted_values_testset['contentId'])
        else:
            person_interacted_items_testset = set([int(interacted_values_testset['contentId'])])
        interacted_items_count_testset = len(person_interacted_items_testset)

        #Getting a ranked recommendation list from a model for a given user
        person_recs_df = model.recommend_items(person_id,
                                               items_to_ignore=get_items_interacted(person_id, # ignore the items from training dataset, we need only brand new items for this person_id
                                                                                    interactions_train_indexed_df),
                                               topn=10000000000)

        hits_at_5_count = 0
        hits_at_10_count = 0
        #For each item the user has interacted in test set
        for item_id in person_interacted_items_testset:
            #Getting a random sample (100) items the user has not interacted
            #(to represent items that are assumed to be no relevant to the user)
            non_interacted_items_sample = self.get_not_interacted_items_sample(person_id,
                                                                          sample_size=EVAL_RANDOM_SAMPLE_NON_INTERACTED_ITEMS,
                                                                          seed=item_id%(2**32))

            #Combining the current interacted item with the 100 random items
            items_to_filter_recs = non_interacted_items_sample.union(set([item_id]))

            #Filtering only recommendations that are either the interacted item or from a random sample of 100 non-interacted items
            valid_recs_df = person_recs_df[person_recs_df['contentId'].isin(items_to_filter_recs)]
            valid_recs = valid_recs_df['contentId'].values #returns list
            #Verifying if the current interacted item is among the Top-N recommended items
            hit_at_5, index_at_5 = self._verify_hit_top_n(item_id, valid_recs, 5)
            hits_at_5_count += hit_at_5
            hit_at_10, index_at_10 = self._verify_hit_top_n(item_id, valid_recs, 10)
            hits_at_10_count += hit_at_10

        #Recall is the rate of the interacted items that are ranked among the Top-N recommended items,
        #when mixed with a set of non-relevant items
        recall_at_5 = hits_at_5_count / float(interacted_items_count_testset)
        recall_at_10 = hits_at_10_count / float(interacted_items_count_testset)

        person_metrics = {'hits@5_count':hits_at_5_count,
                          'hits@10_count':hits_at_10_count,
                          'interacted_count': interacted_items_count_testset,
                          'recall@5': recall_at_5,
                          'recall@10': recall_at_10}
        return person_metrics

    def evaluate_model(self, model):
        #print('Running evaluation for users')
        people_metrics = []
        for idx, person_id in enumerate(list(interactions_test_indexed_df.index.unique().values)):
            #if idx % 100 == 0 and idx > 0:
            #    print('%d users processed' % idx)
            person_metrics = self.evaluate_model_for_user(model, person_id)
            person_metrics['_person_id'] = person_id
            people_metrics.append(person_metrics)
        print('%d users processed' % idx)

        detailed_results_df = pd.DataFrame(people_metrics) \
                            .sort_values('interacted_count', ascending=False)

        global_recall_at_5 = detailed_results_df['hits@5_count'].sum() / float(detailed_results_df['interacted_count'].sum())
        global_recall_at_10 = detailed_results_df['hits@10_count'].sum() / float(detailed_results_df['interacted_count'].sum())

        global_metrics = {'modelName': model.get_model_name(),
                          'recall@5': global_recall_at_5,
                          'recall@10': global_recall_at_10}
        return global_metrics, detailed_results_df

    def dummy():
      pass

model_evaluator = ModelEvaluator()
ModelEvaluator.dummy()

In [90]:
class Model:
  def __init__(self):
    pass
  def recommend_items(self, person_id, items_to_ignore, topn):
    K = 5
    #TODO - items_to_ignore
    users = {v: k for k, v in users_with_enough_interactions_df.to_dict()['personId'].items()}

    articles1 = interactions_full_df.groupby('contentId').size().reset_index()['contentId']
    articles2 = articles_df['contentId']
    articles_full = set(pd.concat([articles1, articles2]))
    articles = {k: v for k, v in zip(articles_full, range(0, len(articles_full)))}

    rating_vectors = np.full((len(users_with_enough_interactions_df), len(articles_full)), float(0))

    for interaction in interactions_full_df.itertuples(index=False):
      rating_vectors[users[interaction.personId]][articles[interaction.contentId]] += interaction.eventStrength

    cosine_dist = {k: v for (k, v) in zip(users.keys(), np.full(len(users), 1))}

    for k, v in users.items():
      if k != person_id:
        cosine_dist[k] = spatial.distance.cosine(rating_vectors[users[person_id]], rating_vectors[v])

    nearest_neighbours = sorted(cosine_dist, key=cosine_dist.get)[0:K]

    v = np.full((K, len(articles_full)), float(1))
    for i in range(K):
      v[i] = rating_vectors[users[nearest_neighbours[i]]]
    v = np.mean(v, axis=0)

    items = np.argsort(v)[::-1]

    recommended_items = [];
    articles_final = {k:v for (v, k) in articles.items()}
    articles_final[1032]
    for i in items:
      recommended_items.append(articles_final[i])

    return pd.Series(recommended_items[:min(topn, len(recommended_items))], name='contentId').to_frame()

In [94]:
mod = Model()
model_evaluator.evaluate_model_for_user(mod, -6411371239765052331)

<ipython-input-92-b9f8eda0cc69>:16: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  non_interacted_items_sample = random.sample(non_interacted_items, sample_size)


{'hits@5_count': 1,
 'hits@10_count': 1,
 'interacted_count': 3,
 'recall@5': 0.3333333333333333,
 'recall@10': 0.3333333333333333}